# Build a Simple LLM Application with LCEL

In [1]:
# import getpass
import os

# using langsmith
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

# using v2rayn
os.environ["HTTP_PROXY"] = "http://127.0.0.1:10809"
os.environ["HTTP_PROXYS"] = "http://127.0.0.1:10809"

# AI model
model = "gemini-1.5-pro"

# conversation session
config = {"configurable": {"session_id": "abc2"}}

In [2]:
import textwrap
# from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

chat = ChatGoogleGenerativeAI(model=model,
                             google_api_key=os.getenv("GOOGLE_API_KEY"))

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        # (
        #     "system",
        #     "You are a helpful assistant. Answer all questions to the best of your ability. Answer me in Chinese.",
        # ),
        SystemMessage("You are a helpful assistant. Answer all questions to the best of your ability. Answer me in Chinese."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat

In [5]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history_for_chain,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [6]:
res = chain_with_message_history.invoke(
    {"input": "Translate this sentence from English to French: I love programming."},
    {"configurable": {"session_id": "unused"}},
)
to_markdown(res.content)

Parent run be6428ed-34e6-41f1-a6c9-19619ad607e5 not found for run e8a5c951-3090-496c-ad92-5af327261064. Treating as a root run.


> 法语是“J'aime programmer”。 


In [7]:
res = chain_with_message_history.invoke(
    {"input": "What did I just ask you?"},
    {"configurable": {"session_id": "unused"}},
)
to_markdown(res.content)

Parent run 1f37c7bf-54ca-470a-abf9-caca95ca54c8 not found for run 26380050-e51b-48b6-90f5-5e7226b53005. Treating as a root run.


> 你刚才问我，如何将英语句子“I love programming”翻译成法语。 😊 


In [ ]:
res = chain_with_message_history.invoke(
    {"input": "My name is Zehao, what is your name?"},
    {"configurable": {"session_id": "unused"}},
)
to_markdown(res.content)

In [ ]:
res = chain_with_message_history.invoke(
    {"input": "What did I just ask you?"},
    {"configurable": {"session_id": "unused"}},
)
to_markdown(res.content)